In [1]:
# data manipulation 
import math 
import numpy as np 
import pandas as pd 

# data visualization 
import matplotlib.pyplot as plt 

# data preprocessing 
from sklearn.preprocessing import MinMaxScaler # to normalize the price data 

# Modeling 
import tensorflow as tf 
from keras.models import Sequential # Deep learning library, used for neural networks
from keras.layers import SimpleRNN, LSTM, Dense # Deep learning classes for recurrent and regular densely-connected layers
from tensorflow.keras.optimizers import Adam, SGD 
from keras.wrappers.scikit_learn import KerasRegressor 
from sklearn.model_selection import RandomizedSearchCV
from keras.callbacks import EarlyStopping

# Data evaluation 
from sklearn.metrics import mean_squared_error 

### Import Dataset

In [2]:
data = pd.read_csv('../dataset.csv')
data.index = pd.to_datetime(data.index)
print(data.shape)
data.head()

(551, 11)


,CPIAUCSL_PC1,UNRATE,FEDFUNDS,T10Y2Y,High,Low,SP500 Close,Volume,ADX,RSI,BB width
1976-06-01,5.98131,7.6,5.48,0.801364,102.507728,100.982727,101.766818,1.896500e+07,NaN,NaN,NaN
1976-07-01,5.55556,7.8,5.31,0.981905,104.933810,103.501905,104.200953,1.875048e+07,NaN,NaN,NaN
1976-08-01,5.71956,7.8,5.29,1.137273,103.985909,102.575001,103.291818,1.575818e+07,NaN,NaN,NaN
1976-09-01,5.49451,7.6,5.25,1.178095,106.130951,104.637618,105.453333,1.975000e+07,NaN,NaN,NaN
1976-10-01,5.46448,7.7,5.02,1.427000,102.681905,101.036191,101.885238,1.720905e+07,NaN,NaN,NaN


In [3]:
data.tail()

,CPIAUCSL_PC1,UNRATE,FEDFUNDS,T10Y2Y,High,Low,SP500 Close,Volume,ADX,RSI,BB width
2021-12-01,7.09654,3.9,0.08,0.785000,4701.581854,4642.169611,4674.772772,3.122720e+09,45.523755,85.192588,0.543355
2022-01-01,7.52593,4.0,0.08,0.783500,4619.576025,4528.042017,4573.815454,3.663972e+09,46.122325,78.669780,0.501704
2022-02-01,7.91202,3.8,0.08,0.498947,4473.607345,4392.072625,4435.980494,3.850936e+09,45.583801,70.709690,0.460878
2022-03-01,8.55759,3.6,0.20,0.218261,4424.881645,4351.570015,4391.265264,4.390362e+09,44.777416,68.295438,0.419147
2022-04-01,NaN,NaN,NaN,0.155556,4500.185872,4439.167440,4468.859945,3.565105e+09,44.302466,70.197059,0.389122


### Data Preprocessing

In [6]:
# Feature Selection - Only Close Data
df = data.filter(['SP500 Close'])
data_unscaled = df.shift(-1).values
# print(len(data_unscaled))
# data_unscaled

# train_data_length = math.ceil(len(data_unscaled) * 0.8)
# print(train_data_length)

# Standardization using MinMaxScaler (Scaling data to a range between 0 and 1)
mmscaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = mmscaler.fit_transform(data_unscaled)

In [7]:
# Set the sequence length - this is the timeframe used to make a single prediction
sequence_length = 50

# Prediction Index
index_Close = df.columns.get_loc('SP500 Close')
print(index_Close)

0


In [8]:
# Split the data into training and testing data sets
# As a first step, we get the number of rows to train the model on 80% of the data 
train_data_len = math.ceil(data_scaled.shape[0] * 0.8)
valid_data_len = math.ceil(train_data_len * 0.2)
print(train_data_len, valid_data_len)

# Create the training, validation, and testing data
train_data = data_scaled[0:train_data_len - valid_data_len, :]
valid_data = data_scaled[train_data_len - valid_data_len:train_data_len,:]
test_data = data_scaled[train_data_len:, :]
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

441 89
(352, 1)
(89, 1)
(110, 1)


In [9]:
train_index = df.index[0:train_data_len - valid_data_len]
valid_index = df.index[train_data_len - valid_data_len:train_data_len]
test_index = df.index[train_data_len:]
print(len(train_index))
print(len(valid_index))
print(len(test_index))

352
89
110


If we have an N-dimensional space and evolve over t-time steps, our input representation must be of shape `(num_samples, t, N)`. In other words, the RNN needs data with the format of [samples, time steps, features]. Here, we are building an univariate model on the S&P 500. Using the monthly aggregation, we have 551 observations and only 1 feature with the sequence length initailized to be 50.  

In [10]:
def partition_dataset(sequence_length, df):
    x, y = [], []
    data_len = df.shape[0]
    for i in range(sequence_length, data_len):
        # x has (data_len - sequence_length) number of sequence_length data window 
        x.append(df[i-sequence_length:i,:])  
        # y has the prediction values for validation,  for single-step prediction
        y.append(df[i, index_Close]) 
    
    # Convert the x and y to numpy arrays
    x = np.array(x)
    y = np.array(y)
    return x, y

In [11]:
# Generate training data, validation, and test data
x_train, y_train = partition_dataset(sequence_length, train_data)
x_valid, y_valid = partition_dataset(sequence_length, valid_data)
x_test, y_test = partition_dataset(sequence_length, test_data)

# Print the shapes: the result is: (rows, training_sequence, features) (prediction value, )
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

# Validate that the prediction value and the input match up
# The last close price of the second input sample should equal the first prediction value
print(x_test[1][sequence_length-1][index_Close])
print(y_test[0])

(302, 50, 1) (302,)
(39, 50, 1) (39,)
(60, 50, 1) (60,)
0.511380249717515
0.511380249717515


### Find Model Architecture

In [12]:
# Model with sequence_length Neurons 
neurons = sequence_length

# input shape = sequence_length Timestamps

def single_layer_RNN_model(optimizer='adam', activation='relu', learning_rate=0.01):
    model = Sequential()
    model.add(SimpleRNN(neurons, return_sequences=False, input_shape=(x_train.shape[1], 1)))
    model.add(Dense(1))
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

def single_layer_LSTM_model(optimizer='adam', activation='relu', learning_rate=0.01):
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=False, input_shape=(x_train.shape[1], 1)))
    model.add(Dense(1))
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

def multiple_layer_LSTM_model(optimizer='adam', activation='relu', learning_rate=0.01):
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], 1))) 
    model.add(LSTM(neurons, return_sequences=False)) 
    model.add(Dense(1))
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model
    
def multiple_layer_LSTM_model_plus(optimizer='adam', activation='relu', learning_rate=0.01):
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=True, input_shape=(x_train.shape[1], 1))) 
    model.add(LSTM(neurons, return_sequences=False))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

### Training the model

In [13]:
rnn_model = KerasRegressor(single_layer_RNN_model) 
params = dict(optimizer=['sgd', 'adam'], epochs=[50, 100], batch_size=[16, 32], activation=['relu','tanh'],
             learning_rate=[0.01, 0.001, 0.0005])
random_search = RandomizedSearchCV(rnn_model, params, cv=3)
rnn_model_results = random_search.fit(x_train, y_train, validation_data=(x_valid, y_valid),
                                          callbacks=[EarlyStopping(patience=3)])
print(f"Best: {rnn_model_results.best_score_} using {rnn_model_results.best_params_}")

<ipython-input-13-e209a15f0dae>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  rnn_model = KerasRegressor(single_layer_RNN_model)


Epoch 1/50
7/7 [==============================] - 1s 32ms/step - loss: 0.0134 - root_mean_squared_error: 0.1158 - val_loss: 0.0010 - val_root_mean_squared_error: 0.0317
Epoch 2/50
7/7 [==============================] - 0s 9ms/step - loss: 0.0020 - root_mean_squared_error: 0.0446 - val_loss: 8.0995e-04 - val_root_mean_squared_error: 0.0285
Epoch 3/50
7/7 [==============================] - 0s 8ms/step - loss: 0.0014 - root_mean_squared_error: 0.0380 - val_loss: 9.3765e-04 - val_root_mean_squared_error: 0.0306
Epoch 4/50
7/7 [==============================] - 0s 8ms/step - loss: 9.1375e-04 - root_mean_squared_error: 0.0302 - val_loss: 7.7868e-04 - val_root_mean_squared_error: 0.0279
Epoch 5/50
7/7 [==============================] - 0s 8ms/step - loss: 7.2986e-04 - root_mean_squared_error: 0.0270 - val_loss: 6.8410e-04 - val_root_mean_squared_error: 0.0262
Epoch 6/50
7/7 [==============================] - 0s 8ms/step - loss: 5.4666e-04 - root_mean_squared_error: 0.0234 - val_loss: 8.1841e-

7/7 [==============================] - 0s 7ms/step - loss: 4.0646e-05 - root_mean_squared_error: 0.0064 - val_loss: 6.9489e-04 - val_root_mean_squared_error: 0.0264
Epoch 29/50
7/7 [==============================] - 0s 7ms/step - loss: 3.9462e-05 - root_mean_squared_error: 0.0063 - val_loss: 6.1210e-04 - val_root_mean_squared_error: 0.0247
Epoch 30/50
7/7 [==============================] - 0s 7ms/step - loss: 3.9463e-05 - root_mean_squared_error: 0.0063 - val_loss: 6.6273e-04 - val_root_mean_squared_error: 0.0257
Epoch 31/50
7/7 [==============================] - 0s 7ms/step - loss: 3.9198e-05 - root_mean_squared_error: 0.0063 - val_loss: 6.5970e-04 - val_root_mean_squared_error: 0.0257
Epoch 32/50
7/7 [==============================] - 0s 7ms/step - loss: 3.7117e-05 - root_mean_squared_error: 0.0061 - val_loss: 5.7740e-04 - val_root_mean_squared_error: 0.0240
Epoch 33/50
7/7 [==============================] - 0s 7ms/step - loss: 3.7448e-05 - root_mean_squared_error: 0.0061 - val_loss:

7/7 [==============================] - 0s 7ms/step - loss: 0.0012 - root_mean_squared_error: 0.0351 - val_loss: 0.0025 - val_root_mean_squared_error: 0.0495
Epoch 21/100
7/7 [==============================] - 0s 6ms/step - loss: 0.0011 - root_mean_squared_error: 0.0333 - val_loss: 0.0021 - val_root_mean_squared_error: 0.0460
Epoch 22/100
7/7 [==============================] - 0s 7ms/step - loss: 0.0010 - root_mean_squared_error: 0.0318 - val_loss: 0.0019 - val_root_mean_squared_error: 0.0440
Epoch 23/100
7/7 [==============================] - 0s 6ms/step - loss: 9.5655e-04 - root_mean_squared_error: 0.0309 - val_loss: 0.0018 - val_root_mean_squared_error: 0.0419
Epoch 24/100
7/7 [==============================] - 0s 7ms/step - loss: 9.1088e-04 - root_mean_squared_error: 0.0302 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0403
Epoch 25/100
7/7 [==============================] - 0s 7ms/step - loss: 8.7748e-04 - root_mean_squared_error: 0.0296 - val_loss: 0.0015 - val_root_mean_squ

Epoch 22/100
7/7 [==============================] - 0s 7ms/step - loss: 1.6034e-04 - root_mean_squared_error: 0.0127 - val_loss: 8.7121e-04 - val_root_mean_squared_error: 0.0295
Epoch 23/100
7/7 [==============================] - 0s 7ms/step - loss: 1.4575e-04 - root_mean_squared_error: 0.0121 - val_loss: 8.1954e-04 - val_root_mean_squared_error: 0.0286
Epoch 24/100
7/7 [==============================] - 0s 7ms/step - loss: 1.3425e-04 - root_mean_squared_error: 0.0116 - val_loss: 7.6560e-04 - val_root_mean_squared_error: 0.0277
Epoch 25/100
7/7 [==============================] - 0s 7ms/step - loss: 1.2373e-04 - root_mean_squared_error: 0.0111 - val_loss: 7.1957e-04 - val_root_mean_squared_error: 0.0268
Epoch 26/100
7/7 [==============================] - 0s 6ms/step - loss: 1.1400e-04 - root_mean_squared_error: 0.0107 - val_loss: 6.5535e-04 - val_root_mean_squared_error: 0.0256
Epoch 27/100
7/7 [==============================] - 0s 7ms/step - loss: 1.0798e-04 - root_mean_squared_error: 

...

13/13 [==============================] - 0s 7ms/step - loss: 9.3244e-04 - root_mean_squared_error: 0.0305 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0401
Epoch 86/100
13/13 [==============================] - 0s 7ms/step - loss: 9.2311e-04 - root_mean_squared_error: 0.0304 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0401
Epoch 87/100
13/13 [==============================] - 0s 6ms/step - loss: 9.1359e-04 - root_mean_squared_error: 0.0302 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0397
Epoch 88/100
13/13 [==============================] - 0s 7ms/step - loss: 9.0522e-04 - root_mean_squared_error: 0.0301 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0393
Epoch 89/100
13/13 [==============================] - 0s 7ms/step - loss: 8.9888e-04 - root_mean_squared_error: 0.0300 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0390
Epoch 90/100
13/13 [==============================] - 0s 7ms/step - loss: 8.8964e-04 - root_mean_squared_error: 0.0298 - val_loss: 0.

13/13 [==============================] - 0s 7ms/step - loss: 7.1364e-04 - root_mean_squared_error: 0.0267 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0403
Epoch 38/100
13/13 [==============================] - 0s 7ms/step - loss: 7.0803e-04 - root_mean_squared_error: 0.0266 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0404
Epoch 39/100
13/13 [==============================] - 0s 7ms/step - loss: 7.1685e-04 - root_mean_squared_error: 0.0268 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0406
Epoch 40/100
13/13 [==============================] - 0s 6ms/step - loss: 6.9343e-04 - root_mean_squared_error: 0.0263 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0402
Epoch 41/100
13/13 [==============================] - 0s 6ms/step - loss: 6.8740e-04 - root_mean_squared_error: 0.0262 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0403
Epoch 42/100
13/13 [==============================] - 0s 7ms/step - loss: 6.7964e-04 - root_mean_squared_error: 0.0261 - val_loss: 0.

13/13 [==============================] - 0s 7ms/step - loss: 2.3363e-04 - root_mean_squared_error: 0.0153 - val_loss: 0.0265 - val_root_mean_squared_error: 0.1629
Epoch 31/100
13/13 [==============================] - 0s 7ms/step - loss: 2.3271e-04 - root_mean_squared_error: 0.0153 - val_loss: 0.0263 - val_root_mean_squared_error: 0.1623
Epoch 32/100
13/13 [==============================] - 0s 7ms/step - loss: 2.3217e-04 - root_mean_squared_error: 0.0152 - val_loss: 0.0261 - val_root_mean_squared_error: 0.1616
Epoch 33/100
13/13 [==============================] - 0s 8ms/step - loss: 2.3142e-04 - root_mean_squared_error: 0.0152 - val_loss: 0.0258 - val_root_mean_squared_error: 0.1607
Epoch 34/100
13/13 [==============================] - 0s 7ms/step - loss: 2.3043e-04 - root_mean_squared_error: 0.0152 - val_loss: 0.0257 - val_root_mean_squared_error: 0.1603
Epoch 35/100
13/13 [==============================] - 0s 8ms/step - loss: 2.2977e-04 - root_mean_squared_error: 0.0152 - val_loss: 0.

13/13 [==============================] - 0s 7ms/step - loss: 2.0934e-04 - root_mean_squared_error: 0.0145 - val_loss: 0.0217 - val_root_mean_squared_error: 0.1475
Epoch 77/100
13/13 [==============================] - 0s 6ms/step - loss: 2.0868e-04 - root_mean_squared_error: 0.0144 - val_loss: 0.0217 - val_root_mean_squared_error: 0.1473
Epoch 78/100
13/13 [==============================] - 0s 7ms/step - loss: 2.0853e-04 - root_mean_squared_error: 0.0144 - val_loss: 0.0216 - val_root_mean_squared_error: 0.1471
Epoch 79/100
13/13 [==============================] - 0s 7ms/step - loss: 2.0802e-04 - root_mean_squared_error: 0.0144 - val_loss: 0.0215 - val_root_mean_squared_error: 0.1468
Epoch 80/100
13/13 [==============================] - 0s 7ms/step - loss: 2.0745e-04 - root_mean_squared_error: 0.0144 - val_loss: 0.0214 - val_root_mean_squared_error: 0.1465
Epoch 81/100
13/13 [==============================] - 0s 7ms/step - loss: 2.0709e-04 - root_mean_squared_error: 0.0144 - val_loss: 0.

7/7 [==============================] - 1s 55ms/step - loss: 0.0054 - root_mean_squared_error: 0.0733 - val_loss: 0.0891 - val_root_mean_squared_error: 0.2984
Epoch 2/50
7/7 [==============================] - 0s 7ms/step - loss: 0.0023 - root_mean_squared_error: 0.0482 - val_loss: 0.0697 - val_root_mean_squared_error: 0.2640
Epoch 3/50
7/7 [==============================] - 0s 7ms/step - loss: 0.0016 - root_mean_squared_error: 0.0405 - val_loss: 0.0482 - val_root_mean_squared_error: 0.2196
Epoch 4/50
7/7 [==============================] - 0s 7ms/step - loss: 0.0011 - root_mean_squared_error: 0.0331 - val_loss: 0.0337 - val_root_mean_squared_error: 0.1836
Epoch 5/50
7/7 [==============================] - 0s 7ms/step - loss: 8.4539e-04 - root_mean_squared_error: 0.0291 - val_loss: 0.0253 - val_root_mean_squared_error: 0.1591
Epoch 6/50
7/7 [==============================] - 0s 8ms/step - loss: 6.1251e-04 - root_mean_squared_error: 0.0247 - val_loss: 0.0214 - val_root_mean_squared_error: 0

In [14]:
sl_LSTM_model = KerasRegressor(single_layer_LSTM_model) 
params = dict(optimizer=['sgd', 'adam'], epochs=[50, 100], batch_size=[16, 32], activation=['relu','tanh'],
             learning_rate=[0.01, 0.001, 0.0005])
random_search = RandomizedSearchCV(sl_LSTM_model, params, cv=3)
sl_LSTM_results = random_search.fit(x_train, y_train, validation_data=(x_valid, y_valid),
                                          callbacks=[EarlyStopping(patience=3)])
print(f"Best: {sl_LSTM_results.best_score_} using {sl_LSTM_results.best_params_}")

<ipython-input-14-83ce690538f2>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  sl_LSTM_model = KerasRegressor(single_layer_LSTM_model)


Epoch 1/50
7/7 [==============================] - 2s 102ms/step - loss: 0.0258 - root_mean_squared_error: 0.1606 - val_loss: 0.0186 - val_root_mean_squared_error: 0.1363
Epoch 2/50
7/7 [==============================] - 0s 18ms/step - loss: 0.0038 - root_mean_squared_error: 0.0617 - val_loss: 6.2848e-04 - val_root_mean_squared_error: 0.0251
Epoch 3/50
7/7 [==============================] - 0s 18ms/step - loss: 0.0036 - root_mean_squared_error: 0.0604 - val_loss: 0.0028 - val_root_mean_squared_error: 0.0527
Epoch 4/50
7/7 [==============================] - 0s 18ms/step - loss: 0.0020 - root_mean_squared_error: 0.0445 - val_loss: 0.0061 - val_root_mean_squared_error: 0.0784
Epoch 5/50
4/4 [==============================] - 0s 5ms/step - loss: 0.0032 - root_mean_squared_error: 0.0566
Epoch 1/50
7/7 [==============================] - 2s 68ms/step - loss: 0.0239 - root_mean_squared_error: 0.1546 - val_loss: 0.0246 - val_root_mean_squared_error: 0.1570
Epoch 2/50
7/7 [=======================

Epoch 29/100
13/13 [==============================] - 0s 14ms/step - loss: 0.0052 - root_mean_squared_error: 0.0720 - val_loss: 0.0133 - val_root_mean_squared_error: 0.1153
Epoch 30/100
13/13 [==============================] - 0s 14ms/step - loss: 0.0051 - root_mean_squared_error: 0.0713 - val_loss: 0.0130 - val_root_mean_squared_error: 0.1138
Epoch 31/100
13/13 [==============================] - 0s 14ms/step - loss: 0.0050 - root_mean_squared_error: 0.0707 - val_loss: 0.0126 - val_root_mean_squared_error: 0.1123
Epoch 32/100
13/13 [==============================] - 0s 14ms/step - loss: 0.0049 - root_mean_squared_error: 0.0701 - val_loss: 0.0123 - val_root_mean_squared_error: 0.1109
Epoch 33/100
13/13 [==============================] - 0s 14ms/step - loss: 0.0048 - root_mean_squared_error: 0.0696 - val_loss: 0.0120 - val_root_mean_squared_error: 0.1096
Epoch 34/100
13/13 [==============================] - 0s 14ms/step - loss: 0.0048 - root_mean_squared_error: 0.0691 - val_loss: 0.0118 

13/13 [==============================] - 0s 14ms/step - loss: 0.0041 - root_mean_squared_error: 0.0639 - val_loss: 0.0076 - val_root_mean_squared_error: 0.0869
Epoch 77/100
13/13 [==============================] - 0s 14ms/step - loss: 0.0041 - root_mean_squared_error: 0.0639 - val_loss: 0.0075 - val_root_mean_squared_error: 0.0868
Epoch 78/100
13/13 [==============================] - 0s 13ms/step - loss: 0.0041 - root_mean_squared_error: 0.0639 - val_loss: 0.0075 - val_root_mean_squared_error: 0.0866
Epoch 79/100
13/13 [==============================] - 0s 13ms/step - loss: 0.0041 - root_mean_squared_error: 0.0639 - val_loss: 0.0075 - val_root_mean_squared_error: 0.0865
Epoch 80/100
13/13 [==============================] - 0s 14ms/step - loss: 0.0041 - root_mean_squared_error: 0.0638 - val_loss: 0.0075 - val_root_mean_squared_error: 0.0864
Epoch 81/100
13/13 [==============================] - 0s 13ms/step - loss: 0.0041 - root_mean_squared_error: 0.0638 - val_loss: 0.0074 - val_root_me

Epoch 23/100
13/13 [==============================] - 0s 16ms/step - loss: 0.0113 - root_mean_squared_error: 0.1062 - val_loss: 0.0273 - val_root_mean_squared_error: 0.1653
Epoch 24/100
13/13 [==============================] - 0s 16ms/step - loss: 0.0111 - root_mean_squared_error: 0.1053 - val_loss: 0.0267 - val_root_mean_squared_error: 0.1634
Epoch 25/100
13/13 [==============================] - 0s 14ms/step - loss: 0.0109 - root_mean_squared_error: 0.1045 - val_loss: 0.0261 - val_root_mean_squared_error: 0.1616
Epoch 26/100
13/13 [==============================] - 0s 14ms/step - loss: 0.0108 - root_mean_squared_error: 0.1038 - val_loss: 0.0256 - val_root_mean_squared_error: 0.1599
Epoch 27/100
13/13 [==============================] - 0s 13ms/step - loss: 0.0106 - root_mean_squared_error: 0.1031 - val_loss: 0.0250 - val_root_mean_squared_error: 0.1582
Epoch 28/100
13/13 [==============================] - 0s 14ms/step - loss: 0.0105 - root_mean_squared_error: 0.1025 - val_loss: 0.0245 

...

13/13 [==============================] - 0s 13ms/step - loss: 7.9117e-04 - root_mean_squared_error: 0.0281 - val_loss: 0.0235 - val_root_mean_squared_error: 0.1532
Epoch 47/100
13/13 [==============================] - 0s 13ms/step - loss: 7.9079e-04 - root_mean_squared_error: 0.0281 - val_loss: 0.0234 - val_root_mean_squared_error: 0.1530
Epoch 48/100
13/13 [==============================] - 0s 12ms/step - loss: 7.9031e-04 - root_mean_squared_error: 0.0281 - val_loss: 0.0234 - val_root_mean_squared_error: 0.1530
Epoch 49/100
13/13 [==============================] - 0s 13ms/step - loss: 7.9012e-04 - root_mean_squared_error: 0.0281 - val_loss: 0.0234 - val_root_mean_squared_error: 0.1529
Epoch 50/100
13/13 [==============================] - 0s 12ms/step - loss: 7.8965e-04 - root_mean_squared_error: 0.0281 - val_loss: 0.0233 - val_root_mean_squared_error: 0.1528
Epoch 51/100
13/13 [==============================] - 0s 12ms/step - loss: 7.8940e-04 - root_mean_squared_error: 0.0281 - val_lo

13/13 [==============================] - 0s 13ms/step - loss: 7.8175e-04 - root_mean_squared_error: 0.0280 - val_loss: 0.0228 - val_root_mean_squared_error: 0.1511
Epoch 93/100
13/13 [==============================] - 0s 13ms/step - loss: 7.8134e-04 - root_mean_squared_error: 0.0280 - val_loss: 0.0228 - val_root_mean_squared_error: 0.1511
Epoch 94/100
13/13 [==============================] - 0s 13ms/step - loss: 7.8123e-04 - root_mean_squared_error: 0.0280 - val_loss: 0.0228 - val_root_mean_squared_error: 0.1510
Epoch 95/100
13/13 [==============================] - 0s 13ms/step - loss: 7.8101e-04 - root_mean_squared_error: 0.0279 - val_loss: 0.0228 - val_root_mean_squared_error: 0.1510
Epoch 96/100
13/13 [==============================] - 0s 13ms/step - loss: 7.8104e-04 - root_mean_squared_error: 0.0279 - val_loss: 0.0228 - val_root_mean_squared_error: 0.1510
Epoch 97/100
13/13 [==============================] - 0s 13ms/step - loss: 7.8076e-04 - root_mean_squared_error: 0.0279 - val_lo

7/7 [==============================] - 0s 15ms/step - loss: 0.0057 - root_mean_squared_error: 0.0757 - val_loss: 0.0155 - val_root_mean_squared_error: 0.1246
Epoch 42/50
7/7 [==============================] - 0s 15ms/step - loss: 0.0056 - root_mean_squared_error: 0.0750 - val_loss: 0.0153 - val_root_mean_squared_error: 0.1236
Epoch 43/50
7/7 [==============================] - 0s 15ms/step - loss: 0.0055 - root_mean_squared_error: 0.0744 - val_loss: 0.0150 - val_root_mean_squared_error: 0.1226
Epoch 44/50
7/7 [==============================] - 0s 15ms/step - loss: 0.0054 - root_mean_squared_error: 0.0738 - val_loss: 0.0148 - val_root_mean_squared_error: 0.1217
Epoch 45/50
7/7 [==============================] - 0s 15ms/step - loss: 0.0054 - root_mean_squared_error: 0.0732 - val_loss: 0.0146 - val_root_mean_squared_error: 0.1206
Epoch 46/50
7/7 [==============================] - 0s 16ms/step - loss: 0.0053 - root_mean_squared_error: 0.0726 - val_loss: 0.0143 - val_root_mean_squared_error:

Epoch 39/50
7/7 [==============================] - 0s 16ms/step - loss: 0.0126 - root_mean_squared_error: 0.1124 - val_loss: 0.0306 - val_root_mean_squared_error: 0.1750
Epoch 40/50
7/7 [==============================] - 0s 17ms/step - loss: 0.0125 - root_mean_squared_error: 0.1118 - val_loss: 0.0302 - val_root_mean_squared_error: 0.1737
Epoch 41/50
7/7 [==============================] - 0s 17ms/step - loss: 0.0124 - root_mean_squared_error: 0.1113 - val_loss: 0.0298 - val_root_mean_squared_error: 0.1726
Epoch 42/50
7/7 [==============================] - 0s 19ms/step - loss: 0.0123 - root_mean_squared_error: 0.1108 - val_loss: 0.0294 - val_root_mean_squared_error: 0.1715
Epoch 43/50
7/7 [==============================] - 0s 18ms/step - loss: 0.0122 - root_mean_squared_error: 0.1103 - val_loss: 0.0290 - val_root_mean_squared_error: 0.1704
Epoch 44/50
7/7 [==============================] - 0s 18ms/step - loss: 0.0121 - root_mean_squared_error: 0.1099 - val_loss: 0.0287 - val_root_mean_sq

Epoch 37/50
7/7 [==============================] - 0s 16ms/step - loss: 0.0032 - root_mean_squared_error: 0.0569 - val_loss: 0.0896 - val_root_mean_squared_error: 0.2993
Epoch 38/50
7/7 [==============================] - 0s 15ms/step - loss: 0.0032 - root_mean_squared_error: 0.0566 - val_loss: 0.0892 - val_root_mean_squared_error: 0.2987
Epoch 39/50
7/7 [==============================] - 0s 16ms/step - loss: 0.0032 - root_mean_squared_error: 0.0563 - val_loss: 0.0888 - val_root_mean_squared_error: 0.2980
Epoch 40/50
7/7 [==============================] - 0s 15ms/step - loss: 0.0031 - root_mean_squared_error: 0.0560 - val_loss: 0.0885 - val_root_mean_squared_error: 0.2974
Epoch 41/50
7/7 [==============================] - 0s 16ms/step - loss: 0.0031 - root_mean_squared_error: 0.0557 - val_loss: 0.0881 - val_root_mean_squared_error: 0.2968
Epoch 42/50
7/7 [==============================] - 0s 15ms/step - loss: 0.0031 - root_mean_squared_error: 0.0554 - val_loss: 0.0877 - val_root_mean_sq

In [15]:
ml_LSTM_model = KerasRegressor(multiple_layer_LSTM_model) 
params = dict(optimizer=['sgd', 'adam'], epochs=[50, 100], batch_size=[16, 32], activation=['relu','tanh'],
             learning_rate=[0.01, 0.001, 0.0005])
random_search = RandomizedSearchCV(ml_LSTM_model, params, cv=3)
ml_LSTM_results = random_search.fit(x_train, y_train, validation_data=(x_valid, y_valid),
                                          callbacks=[EarlyStopping(patience=3)])
print(f"Best: {ml_LSTM_results.best_score_} using {ml_LSTM_results.best_params_}")

<ipython-input-15-369a7f03016f>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ml_LSTM_model = KerasRegressor(multiple_layer_LSTM_model)


Epoch 1/50
7/7 [==============================] - 3s 112ms/step - loss: 0.0102 - root_mean_squared_error: 0.1012 - val_loss: 0.0025 - val_root_mean_squared_error: 0.0499
Epoch 2/50
7/7 [==============================] - 0s 26ms/step - loss: 0.0034 - root_mean_squared_error: 0.0587 - val_loss: 0.0061 - val_root_mean_squared_error: 0.0780
Epoch 3/50
7/7 [==============================] - 0s 27ms/step - loss: 0.0023 - root_mean_squared_error: 0.0483 - val_loss: 0.0031 - val_root_mean_squared_error: 0.0560
Epoch 4/50
7/7 [==============================] - 0s 28ms/step - loss: 0.0013 - root_mean_squared_error: 0.0358 - val_loss: 2.3876e-04 - val_root_mean_squared_error: 0.0155
Epoch 5/50
7/7 [==============================] - 0s 28ms/step - loss: 9.8085e-04 - root_mean_squared_error: 0.0313 - val_loss: 0.0018 - val_root_mean_squared_error: 0.0424
Epoch 6/50
7/7 [==============================] - 0s 28ms/step - loss: 7.0526e-04 - root_mean_squared_error: 0.0266 - val_loss: 5.2409e-04 - val_r

Epoch 2/100
7/7 [==============================] - 0s 27ms/step - loss: 0.0022 - root_mean_squared_error: 0.0473 - val_loss: 0.0549 - val_root_mean_squared_error: 0.2342
Epoch 3/100
7/7 [==============================] - 0s 30ms/step - loss: 0.0018 - root_mean_squared_error: 0.0420 - val_loss: 0.0517 - val_root_mean_squared_error: 0.2273
Epoch 4/100
7/7 [==============================] - 0s 31ms/step - loss: 0.0016 - root_mean_squared_error: 0.0403 - val_loss: 0.0496 - val_root_mean_squared_error: 0.2227
Epoch 5/100
7/7 [==============================] - 0s 30ms/step - loss: 0.0016 - root_mean_squared_error: 0.0397 - val_loss: 0.0484 - val_root_mean_squared_error: 0.2200
Epoch 6/100
7/7 [==============================] - 0s 31ms/step - loss: 0.0016 - root_mean_squared_error: 0.0395 - val_loss: 0.0482 - val_root_mean_squared_error: 0.2196
Epoch 7/100
7/7 [==============================] - 0s 31ms/step - loss: 0.0016 - root_mean_squared_error: 0.0395 - val_loss: 0.0472 - val_root_mean_sq

Epoch 9/100
7/7 [==============================] - 0s 30ms/step - loss: 0.0120 - root_mean_squared_error: 0.1094 - val_loss: 0.0300 - val_root_mean_squared_error: 0.1733
Epoch 10/100
7/7 [==============================] - 0s 29ms/step - loss: 0.0112 - root_mean_squared_error: 0.1057 - val_loss: 0.0285 - val_root_mean_squared_error: 0.1687
Epoch 11/100
7/7 [==============================] - 0s 29ms/step - loss: 0.0105 - root_mean_squared_error: 0.1025 - val_loss: 0.0270 - val_root_mean_squared_error: 0.1644
Epoch 12/100
7/7 [==============================] - 0s 29ms/step - loss: 0.0099 - root_mean_squared_error: 0.0994 - val_loss: 0.0257 - val_root_mean_squared_error: 0.1602
Epoch 13/100
7/7 [==============================] - 0s 29ms/step - loss: 0.0093 - root_mean_squared_error: 0.0966 - val_loss: 0.0244 - val_root_mean_squared_error: 0.1561
Epoch 14/100
7/7 [==============================] - 0s 29ms/step - loss: 0.0088 - root_mean_squared_error: 0.0939 - val_loss: 0.0232 - val_root_me

Epoch 57/100
7/7 [==============================] - 0s 30ms/step - loss: 0.0050 - root_mean_squared_error: 0.0704 - val_loss: 0.0094 - val_root_mean_squared_error: 0.0972
Epoch 58/100
7/7 [==============================] - 0s 30ms/step - loss: 0.0049 - root_mean_squared_error: 0.0703 - val_loss: 0.0094 - val_root_mean_squared_error: 0.0971
Epoch 59/100
7/7 [==============================] - 0s 30ms/step - loss: 0.0049 - root_mean_squared_error: 0.0703 - val_loss: 0.0094 - val_root_mean_squared_error: 0.0968
Epoch 60/100
7/7 [==============================] - 0s 30ms/step - loss: 0.0049 - root_mean_squared_error: 0.0703 - val_loss: 0.0093 - val_root_mean_squared_error: 0.0962
Epoch 61/100
7/7 [==============================] - 0s 29ms/step - loss: 0.0049 - root_mean_squared_error: 0.0702 - val_loss: 0.0093 - val_root_mean_squared_error: 0.0962
Epoch 62/100
7/7 [==============================] - 0s 29ms/step - loss: 0.0049 - root_mean_squared_error: 0.0702 - val_loss: 0.0092 - val_root_m

...

Epoch 3/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0204 - root_mean_squared_error: 0.1427 - val_loss: 0.0505 - val_root_mean_squared_error: 0.2248
Epoch 4/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0194 - root_mean_squared_error: 0.1393 - val_loss: 0.0485 - val_root_mean_squared_error: 0.2202
Epoch 5/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0185 - root_mean_squared_error: 0.1362 - val_loss: 0.0465 - val_root_mean_squared_error: 0.2157
Epoch 6/100
13/13 [==============================] - 0s 25ms/step - loss: 0.0177 - root_mean_squared_error: 0.1332 - val_loss: 0.0448 - val_root_mean_squared_error: 0.2117
Epoch 7/100
13/13 [==============================] - 0s 26ms/step - loss: 0.0170 - root_mean_squared_error: 0.1305 - val_loss: 0.0431 - val_root_mean_squared_error: 0.2076
Epoch 8/100
13/13 [==============================] - 0s 26ms/step - loss: 0.0164 - root_mean_squared_error: 0.1279 - val_loss: 0.0415 - val_

13/13 [==============================] - 0s 23ms/step - loss: 0.0095 - root_mean_squared_error: 0.0973 - val_loss: 0.0179 - val_root_mean_squared_error: 0.1337
Epoch 51/100
13/13 [==============================] - 0s 25ms/step - loss: 0.0095 - root_mean_squared_error: 0.0972 - val_loss: 0.0177 - val_root_mean_squared_error: 0.1332
Epoch 52/100
13/13 [==============================] - 0s 25ms/step - loss: 0.0094 - root_mean_squared_error: 0.0971 - val_loss: 0.0176 - val_root_mean_squared_error: 0.1326
Epoch 53/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0094 - root_mean_squared_error: 0.0970 - val_loss: 0.0175 - val_root_mean_squared_error: 0.1322
Epoch 54/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0094 - root_mean_squared_error: 0.0970 - val_loss: 0.0173 - val_root_mean_squared_error: 0.1316
Epoch 55/100
13/13 [==============================] - 0s 24ms/step - loss: 0.0094 - root_mean_squared_error: 0.0969 - val_loss: 0.0172 - val_root_me

13/13 [==============================] - 0s 23ms/step - loss: 0.0091 - root_mean_squared_error: 0.0953 - val_loss: 0.0149 - val_root_mean_squared_error: 0.1220
Epoch 98/100
13/13 [==============================] - 0s 22ms/step - loss: 0.0091 - root_mean_squared_error: 0.0952 - val_loss: 0.0148 - val_root_mean_squared_error: 0.1219
Epoch 99/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0091 - root_mean_squared_error: 0.0952 - val_loss: 0.0148 - val_root_mean_squared_error: 0.1217
Epoch 100/100
7/7 [==============================] - 0s 6ms/step - loss: 0.0021 - root_mean_squared_error: 0.0461
Epoch 1/100
13/13 [==============================] - 3s 73ms/step - loss: 0.0048 - root_mean_squared_error: 0.0690 - val_loss: 0.0786 - val_root_mean_squared_error: 0.2803
Epoch 2/100
13/13 [==============================] - 0s 25ms/step - loss: 0.0045 - root_mean_squared_error: 0.0668 - val_loss: 0.0770 - val_root_mean_squared_error: 0.2776
Epoch 3/100
13/13 [=================

Epoch 44/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0017 - root_mean_squared_error: 0.0413 - val_loss: 0.0537 - val_root_mean_squared_error: 0.2318
Epoch 45/100
13/13 [==============================] - 0s 24ms/step - loss: 0.0017 - root_mean_squared_error: 0.0412 - val_loss: 0.0536 - val_root_mean_squared_error: 0.2315
Epoch 46/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0017 - root_mean_squared_error: 0.0412 - val_loss: 0.0534 - val_root_mean_squared_error: 0.2312
Epoch 47/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0017 - root_mean_squared_error: 0.0411 - val_loss: 0.0533 - val_root_mean_squared_error: 0.2309
Epoch 48/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0017 - root_mean_squared_error: 0.0411 - val_loss: 0.0532 - val_root_mean_squared_error: 0.2306
Epoch 49/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0017 - root_mean_squared_error: 0.0411 - val_loss: 0.0531 

13/13 [==============================] - 0s 23ms/step - loss: 0.0017 - root_mean_squared_error: 0.0407 - val_loss: 0.0510 - val_root_mean_squared_error: 0.2259
Epoch 92/100
13/13 [==============================] - 0s 24ms/step - loss: 0.0017 - root_mean_squared_error: 0.0407 - val_loss: 0.0510 - val_root_mean_squared_error: 0.2259
Epoch 93/100
13/13 [==============================] - 0s 24ms/step - loss: 0.0017 - root_mean_squared_error: 0.0407 - val_loss: 0.0510 - val_root_mean_squared_error: 0.2258
Epoch 94/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0017 - root_mean_squared_error: 0.0407 - val_loss: 0.0510 - val_root_mean_squared_error: 0.2258
Epoch 95/100
13/13 [==============================] - 0s 23ms/step - loss: 0.0017 - root_mean_squared_error: 0.0407 - val_loss: 0.0510 - val_root_mean_squared_error: 0.2258
Epoch 96/100
13/13 [==============================] - 0s 24ms/step - loss: 0.0017 - root_mean_squared_error: 0.0407 - val_loss: 0.0510 - val_root_me

In [16]:
mlp_LSTM_model = KerasRegressor(multiple_layer_LSTM_model_plus) 
params = dict(optimizer=['sgd', 'adam'], epochs=[50, 100], batch_size=[16, 32], activation=['relu','tanh'],
             learning_rate=[0.01, 0.001, 0.0005])
random_search = RandomizedSearchCV(mlp_LSTM_model, params, cv=3)
mlp_LSTM_results = random_search.fit(x_train, y_train, validation_data=(x_valid, y_valid),
                                          callbacks=[EarlyStopping(patience=3)])
print(f"Best: {mlp_LSTM_results.best_score_} using {mlp_LSTM_results.best_params_}")

<ipython-input-16-479eaed71c97>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  mlp_LSTM_model = KerasRegressor(multiple_layer_LSTM_model_plus)


Epoch 1/50
7/7 [==============================] - 3s 113ms/step - loss: 0.0125 - root_mean_squared_error: 0.1119 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0387
Epoch 2/50
7/7 [==============================] - 0s 29ms/step - loss: 0.0033 - root_mean_squared_error: 0.0571 - val_loss: 0.0076 - val_root_mean_squared_error: 0.0873
Epoch 3/50
7/7 [==============================] - 0s 30ms/step - loss: 0.0025 - root_mean_squared_error: 0.0500 - val_loss: 0.0031 - val_root_mean_squared_error: 0.0558
Epoch 4/50
7/7 [==============================] - 0s 31ms/step - loss: 0.0012 - root_mean_squared_error: 0.0341 - val_loss: 2.7936e-04 - val_root_mean_squared_error: 0.0167
Epoch 5/50
7/7 [==============================] - 0s 31ms/step - loss: 8.8166e-04 - root_mean_squared_error: 0.0297 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0413
Epoch 6/50
7/7 [==============================] - 0s 31ms/step - loss: 6.5059e-04 - root_mean_squared_error: 0.0255 - val_loss: 4.9701e-04 - val_r

7/7 [==============================] - 3s 115ms/step - loss: 0.0179 - root_mean_squared_error: 0.1337 - val_loss: 0.0343 - val_root_mean_squared_error: 0.1852
Epoch 2/50
7/7 [==============================] - 0s 28ms/step - loss: 0.0118 - root_mean_squared_error: 0.1084 - val_loss: 0.0245 - val_root_mean_squared_error: 0.1567
Epoch 3/50
7/7 [==============================] - 0s 30ms/step - loss: 0.0097 - root_mean_squared_error: 0.0983 - val_loss: 0.0191 - val_root_mean_squared_error: 0.1382
Epoch 4/50
7/7 [==============================] - 0s 30ms/step - loss: 0.0088 - root_mean_squared_error: 0.0937 - val_loss: 0.0162 - val_root_mean_squared_error: 0.1271
Epoch 5/50
7/7 [==============================] - 0s 33ms/step - loss: 0.0084 - root_mean_squared_error: 0.0917 - val_loss: 0.0144 - val_root_mean_squared_error: 0.1198
Epoch 6/50
7/7 [==============================] - 0s 31ms/step - loss: 0.0082 - root_mean_squared_error: 0.0907 - val_loss: 0.0137 - val_root_mean_squared_error: 0.1

7/7 [==============================] - 0s 32ms/step - loss: 0.0083 - root_mean_squared_error: 0.0911 - val_loss: 0.0218 - val_root_mean_squared_error: 0.1476
Epoch 13/50
7/7 [==============================] - 0s 31ms/step - loss: 0.0078 - root_mean_squared_error: 0.0884 - val_loss: 0.0207 - val_root_mean_squared_error: 0.1439
Epoch 14/50
7/7 [==============================] - 0s 33ms/step - loss: 0.0074 - root_mean_squared_error: 0.0860 - val_loss: 0.0197 - val_root_mean_squared_error: 0.1404
Epoch 15/50
7/7 [==============================] - 0s 34ms/step - loss: 0.0070 - root_mean_squared_error: 0.0838 - val_loss: 0.0188 - val_root_mean_squared_error: 0.1372
Epoch 16/50
7/7 [==============================] - 0s 34ms/step - loss: 0.0067 - root_mean_squared_error: 0.0819 - val_loss: 0.0180 - val_root_mean_squared_error: 0.1342
Epoch 17/50
7/7 [==============================] - 0s 35ms/step - loss: 0.0064 - root_mean_squared_error: 0.0802 - val_loss: 0.0172 - val_root_mean_squared_error:

Epoch 10/50
7/7 [==============================] - 0s 31ms/step - loss: 0.0144 - root_mean_squared_error: 0.1200 - val_loss: 0.0365 - val_root_mean_squared_error: 0.1910
Epoch 11/50
7/7 [==============================] - 0s 30ms/step - loss: 0.0139 - root_mean_squared_error: 0.1179 - val_loss: 0.0352 - val_root_mean_squared_error: 0.1877
Epoch 12/50
7/7 [==============================] - 0s 31ms/step - loss: 0.0135 - root_mean_squared_error: 0.1160 - val_loss: 0.0341 - val_root_mean_squared_error: 0.1845
Epoch 13/50
7/7 [==============================] - 0s 35ms/step - loss: 0.0131 - root_mean_squared_error: 0.1143 - val_loss: 0.0328 - val_root_mean_squared_error: 0.1812
Epoch 14/50
7/7 [==============================] - 0s 34ms/step - loss: 0.0127 - root_mean_squared_error: 0.1126 - val_loss: 0.0317 - val_root_mean_squared_error: 0.1781
Epoch 15/50
7/7 [==============================] - 0s 33ms/step - loss: 0.0123 - root_mean_squared_error: 0.1111 - val_loss: 0.0307 - val_root_mean_sq

...

Epoch 26/100
13/13 [==============================] - 0s 30ms/step - loss: 0.0015 - root_mean_squared_error: 0.0382 - val_loss: 0.0459 - val_root_mean_squared_error: 0.2142
Epoch 27/100
13/13 [==============================] - 0s 29ms/step - loss: 0.0015 - root_mean_squared_error: 0.0381 - val_loss: 0.0457 - val_root_mean_squared_error: 0.2138
Epoch 28/100
13/13 [==============================] - 0s 28ms/step - loss: 0.0015 - root_mean_squared_error: 0.0381 - val_loss: 0.0455 - val_root_mean_squared_error: 0.2134
Epoch 29/100
13/13 [==============================] - 0s 28ms/step - loss: 0.0014 - root_mean_squared_error: 0.0380 - val_loss: 0.0454 - val_root_mean_squared_error: 0.2130
Epoch 30/100
13/13 [==============================] - 0s 28ms/step - loss: 0.0014 - root_mean_squared_error: 0.0380 - val_loss: 0.0452 - val_root_mean_squared_error: 0.2127
Epoch 31/100
13/13 [==============================] - 0s 29ms/step - loss: 0.0014 - root_mean_squared_error: 0.0380 - val_loss: 0.0451 

13/13 [==============================] - 0s 27ms/step - loss: 0.0014 - root_mean_squared_error: 0.0377 - val_loss: 0.0436 - val_root_mean_squared_error: 0.2088
Epoch 74/100
13/13 [==============================] - 0s 28ms/step - loss: 0.0014 - root_mean_squared_error: 0.0378 - val_loss: 0.0436 - val_root_mean_squared_error: 0.2088
Epoch 75/100
13/13 [==============================] - 0s 27ms/step - loss: 0.0014 - root_mean_squared_error: 0.0378 - val_loss: 0.0436 - val_root_mean_squared_error: 0.2088
Epoch 76/100
13/13 [==============================] - 0s 27ms/step - loss: 0.0014 - root_mean_squared_error: 0.0377 - val_loss: 0.0436 - val_root_mean_squared_error: 0.2088
Epoch 77/100
13/13 [==============================] - 0s 27ms/step - loss: 0.0014 - root_mean_squared_error: 0.0377 - val_loss: 0.0436 - val_root_mean_squared_error: 0.2088
Epoch 78/100
13/13 [==============================] - 0s 26ms/step - loss: 0.0014 - root_mean_squared_error: 0.0377 - val_loss: 0.0436 - val_root_me

13/13 [==============================] - 0s 26ms/step - loss: 0.0035 - root_mean_squared_error: 0.0593 - val_loss: 0.0069 - val_root_mean_squared_error: 0.0830
Epoch 25/100
13/13 [==============================] - 0s 26ms/step - loss: 0.0034 - root_mean_squared_error: 0.0585 - val_loss: 0.0059 - val_root_mean_squared_error: 0.0768
Epoch 26/100
13/13 [==============================] - 0s 25ms/step - loss: 0.0033 - root_mean_squared_error: 0.0577 - val_loss: 0.0055 - val_root_mean_squared_error: 0.0740
Epoch 27/100
13/13 [==============================] - 0s 27ms/step - loss: 0.0032 - root_mean_squared_error: 0.0570 - val_loss: 0.0057 - val_root_mean_squared_error: 0.0755
Epoch 28/100
13/13 [==============================] - 0s 25ms/step - loss: 0.0032 - root_mean_squared_error: 0.0563 - val_loss: 0.0059 - val_root_mean_squared_error: 0.0768
Epoch 29/100
7/7 [==============================] - 0s 6ms/step - loss: 5.8695e-04 - root_mean_squared_error: 0.0242
Epoch 1/100
13/13 [============

Epoch 4/100
7/7 [==============================] - 0s 33ms/step - loss: 1.9122e-04 - root_mean_squared_error: 0.0138 - val_loss: 4.4191e-04 - val_root_mean_squared_error: 0.0210
Epoch 5/100
7/7 [==============================] - 0s 32ms/step - loss: 1.1293e-04 - root_mean_squared_error: 0.0106 - val_loss: 8.3292e-04 - val_root_mean_squared_error: 0.0289
Epoch 6/100
7/7 [==============================] - 0s 33ms/step - loss: 7.8854e-05 - root_mean_squared_error: 0.0089 - val_loss: 3.1886e-04 - val_root_mean_squared_error: 0.0179
Epoch 7/100
7/7 [==============================] - 0s 32ms/step - loss: 6.6443e-05 - root_mean_squared_error: 0.0082 - val_loss: 1.9386e-04 - val_root_mean_squared_error: 0.0139
Epoch 8/100
7/7 [==============================] - 0s 32ms/step - loss: 5.3900e-05 - root_mean_squared_error: 0.0073 - val_loss: 7.7486e-04 - val_root_mean_squared_error: 0.0278
Epoch 9/100
7/7 [==============================] - 0s 32ms/step - loss: 4.5716e-05 - root_mean_squared_error: 

Epoch 12/100
7/7 [==============================] - 0s 35ms/step - loss: 0.0051 - root_mean_squared_error: 0.0715 - val_loss: 0.0093 - val_root_mean_squared_error: 0.0963
Epoch 13/100
4/4 [==============================] - 0s 8ms/step - loss: 0.0135 - root_mean_squared_error: 0.1163
Epoch 1/100
7/7 [==============================] - 3s 158ms/step - loss: 0.0209 - root_mean_squared_error: 0.1446 - val_loss: 0.0362 - val_root_mean_squared_error: 0.1904
Epoch 2/100
7/7 [==============================] - 0s 32ms/step - loss: 0.0118 - root_mean_squared_error: 0.1086 - val_loss: 0.0242 - val_root_mean_squared_error: 0.1556
Epoch 3/100
7/7 [==============================] - 0s 34ms/step - loss: 0.0091 - root_mean_squared_error: 0.0955 - val_loss: 0.0192 - val_root_mean_squared_error: 0.1387
Epoch 4/100
7/7 [==============================] - 0s 34ms/step - loss: 0.0081 - root_mean_squared_error: 0.0901 - val_loss: 0.0153 - val_root_mean_squared_error: 0.1239
Epoch 5/100
7/7 [==================

### Best Model 

In [17]:
rnn = round(abs(rnn_model_results.best_score_), 6)
sl = round(abs(sl_LSTM_results.best_score_), 6)
ml = round(abs(ml_LSTM_results.best_score_), 6)
mlp = round(abs(mlp_LSTM_results.best_score_), 6)

print(f"The Root Mean Squared Error for single layer RNN model is {rnn}")
print(f"The Root Mean Squared Error for single layer LSTM model is {sl}")
print(f"The Root Mean Squared Error for two layers LSTM model is {ml}")
print(f"The Root Mean Squared Error for two layers LSTM model with a dense layer is {mlp}")

The Root Mean Squared Error for single layer RNN model is 0.000213
The Root Mean Squared Error for single layer LSTM model is 0.000146
The Root Mean Squared Error for two layers LSTM model is 0.000433
The Root Mean Squared Error for two layers LSTM model with a dense layer is 0.000442


In [18]:
sl_LSTM_results.best_params_

{'optimizer': 'adam',
 'learning_rate': 0.01,
 'epochs': 100,
 'batch_size': 32,
 'activation': 'relu'}

In [21]:
# Configure the neural network model
model = Sequential()

neurons = sequence_length 

# Model with sequence_length Neurons 
# inputshape = sequence_length Timestamps 
model.add(LSTM(neurons, activation="relu", return_sequences=False, input_shape=(x_train.shape[1], 1))) 
model.add(Dense(1)) 

optimizer = Adam(learning_rate=0.01) 
# Compile the model 
model.compile(optimizer='adam', loss='mean_squared_error') 

# Training the model 
model.fit(x_train, y_train, batch_size=32, epochs=100) 

Epoch 1/100
10/10 [==============================] - 1s 13ms/step - loss: 0.0092
Epoch 2/100
10/10 [==============================] - 0s 13ms/step - loss: 0.0026
Epoch 3/100
10/10 [==============================] - 0s 13ms/step - loss: 0.0017
Epoch 4/100
10/10 [==============================] - 0s 14ms/step - loss: 8.5511e-04
Epoch 5/100
10/10 [==============================] - 0s 13ms/step - loss: 4.1130e-04
Epoch 6/100
10/10 [==============================] - 0s 13ms/step - loss: 3.2282e-04
Epoch 7/100
10/10 [==============================] - 0s 13ms/step - loss: 2.7647e-04
Epoch 8/100
10/10 [==============================] - 0s 13ms/step - loss: 2.6408e-04
Epoch 9/100
10/10 [==============================] - 0s 13ms/step - loss: 2.3805e-04
Epoch 10/100
10/10 [==============================] - 0s 13ms/step - loss: 2.3008e-04
Epoch 11/100
10/10 [==============================] - 0s 13ms/step - loss: 2.1493e-04
Epoch 12/100
10/10 [==============================] - 0s 12ms/step - loss: 

10/10 [==============================] - 0s 14ms/step - loss: 5.6739e-05
Epoch 97/100
10/10 [==============================] - 0s 14ms/step - loss: 6.0651e-05
Epoch 98/100
10/10 [==============================] - 0s 13ms/step - loss: 5.8581e-05
Epoch 99/100
10/10 [==============================] - 0s 13ms/step - loss: 6.3481e-05
Epoch 100/100
10/10 [==============================] - 0s 14ms/step - loss: 5.7750e-05


In [22]:
model.save('saved_model/uni_model')

INFO:tensorflow:Assets written to: saved_model/uni_model\assets


INFO:tensorflow:Assets written to: saved_model/uni_model\assets
